In [1]:
from data import *

Using TensorFlow backend.


# Data Augmentasi

Pembuatan data augmentasi menggunakan library **keras.preprocessing.image.ImageDataGenerator** yang bisa menggabungkan Deep Neural Network image dan label secara bersamaan.

## Tentukan parameter data generator
Gunakan parameter dengan metode random dictionary seperti dibawah ini.

In [2]:
data_gen_args = dict(rotation_range = 0.05,
                    width_shift_range = 0.01,
                    height_shift_range = 0.01,
                    shear_range = 0.05,
                    zoom_range = 0.05,
                    horizontal_flip = True,
                    fill_mode = 'nearest')
myGenerator = trainGenerator(600, 'data/train', 'image', 'label', data_gen_args, save_to_dir = "data/train/aug")

## Membuat data augmentasi ke folder **'data/train/aug/'**

In [3]:
num_batch = 2
for i, batch in enumerate(myGenerator):
    print(i)
    if(i>=(num_batch-1)):
        break

Found 600 images belonging to 1 classes.
Found 600 images belonging to 1 classes.
0
1


## Membuat .npy data (data augmentasi yang sudah dikompres)
Butuh memori tambahan untuk menyimpan data ini

In [4]:
image_arr,mask_arr = geneTrainNpy("data/train/aug/","data/train/aug/")
np.save("data/image_arr.npy",image_arr)
np.save("data/mask_arr.npy",mask_arr)

## Masking Test Data

In [2]:
def masking(img, mask, height=512, width=470, color = yellow): #GE 432 532
    '''.'''
    mask_out = np.zeros((512, 470, 3), dtype = 'uint8') #phantom
    #mask_out = np.zeros((height, width, 3), dtype = 'uint8') #GE
    #img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) #kalau shapenya (432,532) atau grayscale 
    img = np.zeros((height, width, 3), dtype = 'uint8') 
    for i in range(mask.shape[0]-1):
        for j in range(mask.shape[1]-1):
            if (mask[i,j] >= 30):
                mask_out[i,j,:] = mask[i,j] 
                mask_out[i,j,:] = color
                img[i,j,:] = mask_out[i,j,:]
    segmented = img
    return segmented

In [4]:
import os
from os import listdir
from skimage import io
from IPython.display import clear_output

results_path = 'data/test-phantom/'
results_results = 'results/'
list_file = os.listdir(results_path)
masked = np.zeros((len(list_file)//2, 512, 470, 3), dtype ='uint8') #GE 432 532

for i in range(len(list_file)//2):
    #masked[i] = masking(np.zeros((432, 532, 3), dtype = 'uint8'), io.imread(results_path+str(i)+'_predict.png'), color = white)
    masked[i] = masking(io.imread(results_path+str(i)+'.png'), io.imread(results_path+str(i)+'_predict.png'), color = white)
    if (i % 10 == 0):
        clear_output(wait=True)
    else:
        print('Creating... ' + results_path+str(i)+'_predict.png')

    io.imsave(os.path.join(results_results,"%d.png"%i), masked[i])

Creating... data/test-phantom/161_predict.png
Creating... data/test-phantom/162_predict.png
Creating... data/test-phantom/163_predict.png
Creating... data/test-phantom/164_predict.png
Creating... data/test-phantom/165_predict.png
Creating... data/test-phantom/166_predict.png


# CARA CONVERT PNG TO MP4

In [ ]:
ffmpeg -r 30 -f image2 -i %d.png -vcodec libx264 -crf 15  -pix_fmt yuv420p test.mp4

# CARA MENGGABUNGKAN 2 VIDEO YANG SAMA

In [ ]:
ffmpeg -i Yoga60.avi -i aqua.mp4 -filter_complex "[0:v]pad=iw*2:ih[int]; [int][1:v]overlay=W/2:0[vid]" -map "[vid]" -c:v libx264 -crf 15 output2.mp4

## DICE SIMILARITY COEFFICIENT

In [2]:
def dice(im1, im2, empty_score=1.0):
    """
    Computes the Dice coefficient, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    dice : float
        Dice coefficient as a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0
        Both are empty (sum eq to zero) = empty_score
        
    Notes
    -----
    The order of inputs for `dice` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    im_sum = im1.sum() + im2.sum()
    if im_sum == 0:
        return empty_score

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / im_sum

In [4]:
import os
from os import listdir
from skimage import io
from IPython.display import clear_output

path_predict = './results/result threshold 220/'
path_acuan = './data/Ground Truth/'

list_predict = os.listdir(path_predict)
list_acuan = os.listdir(path_acuan)
dsc = np.zeros(len(list_predict), dtype = float)
temp = 0

for i in range(len(list_predict)):
    img1 = cv2.cvtColor(cv2.imread(path_predict+str(i)+'.png'), cv2.COLOR_RGB2GRAY)
    img2 = cv2.cvtColor(cv2.imread(path_acuan+str(i)+'.png'), cv2.COLOR_RGB2GRAY)
    dsc[i] = dice(img1, img2)
    temp = temp + dsc[i]
    print(i, dsc[i])
hasil = temp/len(list_predict)
print()
print(hasil)

0 0.5786037167252637
1 0.568561872909699
2 0.5670654110460677
3 0.6936148300720907
4 0.78994708994709
5 0.8595431871882384
6 0.9094240837696335
7 0.9388180161416297
8 0.9383047210300429
9 0.9394998655552568
10 0.9530799475753604
11 0.9403973509933775
12 0.9393787575150301
13 0.9341603053435115
14 0.9453049370764763
15 0.94018296973962
16 0.9422521655437921
17 0.9504015115729806
18 0.9448893075836081
19 0.9483793517406963
20 0.9414906530711338
21 0.9557781578304422
22 0.9395672718229297
23 0.9275981832754475
24 0.9435870143693454
25 0.9448147160757131
26 0.9429413340476829
27 0.9502053388090349
28 0.9129389129389129
29 0.9185875429131928
30 0.9069432440218957
31 0.9380795692491773
32 0.8997059784384188
33 0.8677711784932791
34 0.8244416873449132
35 0.9136913129041327
36 0.9213071895424837
37 0.9251170046801872
38 0.9168621318738598
39 0.893143596377749
40 0.8792517006802721
41 0.9308909242298085
42 0.912549687677456
43 0.9313609467455621
44 0.9546682677874512
45 0.9398092967818832
46 0.

## JACCARD SIMILARITY COEFFICIENT

In [16]:
import numpy as np


def jaccard(im1, im2):
    """
    Computes the Jaccard metric, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    jaccard : float
        Jaccard metric returned is a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0
    
    Notes
    -----
    The order of inputs for `jaccard` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")
    
    intersection = np.logical_and(im1, im2)

    union = np.logical_or(im1, im2)

    return intersection.sum() / float(union.sum())

In [17]:
import os
from os import listdir
from skimage import io
from IPython.display import clear_output

path_predict = './results/result phantom/'
path_acuan = './data/Ground Truth Phantom/'

list_predict = os.listdir(path_predict)
list_acuan = os.listdir(path_acuan)
dsc = np.zeros(len(list_predict), dtype = float)
temp = 0

for i in range(len(list_predict)-1):
    img1 = cv2.cvtColor(cv2.imread(path_predict+str(i)+'.png'), cv2.COLOR_RGB2GRAY)
    img2 = cv2.cvtColor(cv2.imread(path_acuan+str(i)+'.png'), cv2.COLOR_RGB2GRAY)
    dsc[i] = jaccard(img1, img2)
    temp = temp + dsc[i]
    #print(dsc[i])
hasil = temp/len(list_predict)
print(hasil)

0.9371651545218203


## PIXEL DIFFERENCE

In [23]:
import os
from os import listdir
from skimage import io
import cv2
import numpy as np 
import matplotlib.pyplot as plt 

path_predict = './results/result phantom/'
path_acuan = './data/Ground Truth Phantom/'

list_predict = os.listdir(path_predict)
list_acuan = os.listdir(path_acuan)
temp = 0
aa = 0
bb = 0
for i in range(len(list_acuan)):
    a = np.count_nonzero(cv2.imread(path_predict+str(i)+'.png')==255)
    b = np.count_nonzero(cv2.imread(path_acuan+str(i)+'.png')==255)
    avg = abs(a-b)/b
    print(i, b, a, avg)
    temp = temp + avg
    aa = aa + a
    bb = bb + b
temp = temp/len(list_acuan)
aa = aa/len(list_acuan)
bb = bb/len(list_acuan)
print()
print(aa, bb, temp)

0 14259 14751 0.034504523458868085
1 14181 14688 0.03575206261899725
2 14202 14673 0.03316434305027461
3 14025 14475 0.03208556149732621
4 13905 14415 0.036677454153182305
5 14403 14763 0.024994792751510102
6 14454 14916 0.0319634703196347
7 13692 14025 0.02432077125328659
8 13596 14130 0.03927625772285966
9 13641 14268 0.045964372113481415
10 13716 14016 0.021872265966754154
11 13488 14325 0.062055160142348756
12 12930 13356 0.032946635730858466
13 12783 13176 0.030743956817648438
14 12741 13161 0.03296444549093478
15 12942 13284 0.02642559109874826
16 12816 13251 0.03394194756554307
17 12726 13089 0.028524280999528525
18 12741 13074 0.026136096067812573
19 12726 13053 0.025695426685525696
20 12834 13293 0.03576437587657784
21 12828 13179 0.02736202057998129
22 12912 13320 0.031598513011152414
23 13050 13476 0.03264367816091954
24 13320 13737 0.031306306306306306
25 13485 13944 0.03403781979977753
26 13239 13659 0.031724450487196916
27 13197 13572 0.028415548988406456
28 13509 13884 0